In [58]:
import pandas as pd

In [61]:
df = pd.read_parquet('tiling_results.parquet')

In [62]:
df.shape

(1400, 12)

In [48]:
len(df[df.tiling_compatible == True]) 

142

In [49]:
len(df[df.incompatible_reason == 'unsupported_format'])

359

In [50]:
len(df[df.incompatible_reason == 'cant_open_file'])

407

In [51]:
142 +359 + 407

908